In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("assignment7.ipynb")

# Applied Neural Networks using PyTorch

This assignment aims to give you practical experience in building, training, and evaluating neural networks using PyTorch. You'll apply neural networks to a real-world dataset and gain hands-on experience with industry-standard tools and techniques.

[PyTorch](https://pytorch.org/) is an open-source machine learning library developed by Facebook's AI Research lab. It provides a flexible and efficient platform for building and training neural networks. It supports all kinds of applications: computer vision, natural language processing, and reinforcement learning. PyTorch is an extremely popular choice for both research and production environments.

## Part 1 

## Data Preparation

In [ ]:
# Importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

We'll be using the [CIFAR-100 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). It consists of 60,000 32x32 color images divided into 100 different classes, with 600 images per class. The dataset is split into 50,000 training images and 10,000 test images. The 100 classes are grouped into 20 superclasses. For example, the superclass "fish" contains classes like "aquarium fish", "flatfish", "ray", "shark", and "trout". This makes CIFAR-100 more challenging than similar datasets like CIFAR-10, as it requires models to make more fine-grained distinctions between classes.

PyTorch has a library called `torchvision` that provides a comprehensive set of tools for working with images and videos. We can load CIFAR-100 (and others!) using `torchvision.datasets`. 

We're also defining some transformations that will be performed on the images when loading the dataset. 

**Question 1:** Your first task two-fold: 

1. Define the `training_set` and `validation_set` variables to the appropriate splits of the CIFAR-100 dataset. Don't forget to apply the transforms. Use [this documentation page](https://pytorch.org/vision/main/generated/torchvision.datasets.CIFAR100.html) for reference.
2. Define the `training_loader` and `validation_loader` varibles to dataloaders for the two datasets we instantiated before. Provide a batch size of 64.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Note that this will download the dataset into a directory called 'data' in your current working directory
training_set = ...
validation_set = ...

# To simplify the data loading, we will use a DataLoader object provided by PyTorch with a batch size of 64
training_loader = ...
validation_loader = ...

In [ ]:
grader.check("q1")

### Data Exploration

In [ ]:
print(f"Training set has {len(training_set)} images")
print(f"Validation set has {len(validation_set)} images")

In [ ]:
print(f"Each image is of size {training_set[0][0].size()}")

You can view the classes using `.classes`. 

In [ ]:
classes = training_set.classes
print(f"Classes: {classes}")
print(f"Number of classes: {len(classes)}")

Remember, the `training_loader` is defined as an iterable that returns images in
batches. We can see how many iterations are required to return unique batches of
our training dataset.

In [ ]:
print(f"Training loader has {len(training_loader)} batches")

Let's look at some of the images.  You can re-run this cell to look at a different batch.

In [ ]:
images, labels = next(iter(training_loader)) # This loads a batch of images and their labels

# Display the images
fig, axs = plt.subplots(1, 8, figsize=(20, 5))
for i, ax in enumerate(axs):
    ax.imshow(images[i].permute(1, 2, 0))
    ax.set_title(classes[labels[i]])
    ax.axis('off')
    
plt.show()

We can also look at the histograms of each channel of the batch of images as well
as compute the mean and standard deviation.

In [ ]:
import matplotlib.pyplot as plt

images, labels = next(iter(training_loader))

# Assuming `images` is a batch of images with shape (batch_size, channels, height, width)
batch_size, channels, height, width = images.shape

# Compute mean and std for each channel
mean = images.mean(dim=[0, 2, 3])
std = images.std(dim=[0, 2, 3])

print(f"Mean: {mean}")
print(f"Std: {std}")

# Plot histograms for each channel
fig, axs = plt.subplots(1, channels, figsize=(15, 5))
for i in range(channels):
    axs[i].hist(images[:, i, :, :].flatten().numpy(), bins=50, color='blue', alpha=0.7)
    axs[i].set_title(f'Channel {i+1} Histogram')
    axs[i].set_xlabel('Pixel Values')
    axs[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## Part 2 

## Building and Training a Convolutional Neural Network

In this part, we will build a Convolutional Neural Network (CNN) from scratch to classify the images in our dataset. CNNs are particularly well-suited for image classification tasks because they can automatically learn hierarchical features from the input images through their convolutional layers.
# 
We will:
1. Define a CNN architecture with convolutional layers, pooling layers, and fully connected layers
2. Train the model using our training data
3. Evaluate the model's performance on the test set

The CNN will need to learn to distinguish between our 100 different classes of images by identifying relevant features and patterns in the input data.


**Question 2:** Define a class called `CNN` which we will use to instantiate our model. Use the `torch.nn` module to define it's layers and the `torch.nn.functional` library to define activations. Your model should have at least one convolutional layer. You are free to define the rest of the layers model and activations - for example, you could have a series of convolutional layers followed by a series of linear layers. We encourage you to experiment and get your prediction accuracy as high as possible! 

Use [the course notes on CNNs](https://tools4ds.github.io/DS701-Course-Notes/25-NN-III-CNNs.html#define-a-cnn-in-pytorch), [the discussion notebook on neural networks](https://github.com/tools4ds/ds701_fa2024_assignments/blob/main/discussions/discussion11/discussion11-NN.ipynb) and PyTorch documentation for reference. 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        ...
        
    def forward(self, x):

        ...
        
        return x

# Create an instance of the CNN
model = CNN()
print(model)


In [ ]:
grader.check("q2")

**Question 3**: Define the optimizer and the loss function. Pick what's appropriate from PyTorch documentation. 

In [ ]:
optimizer = ...
loss_fn = ...

In [ ]:
grader.check("q3")

<!-- BEGIN QUESTION -->

**Question 4:** Train the model for at least 10 epochs, and plot the performance. Define a function `train_model` that takes the training and validation dataloaders, optimizer, loss function, and the number of epochs to train.

The function should return the training losses and validation accuracies. 

The training loop is as follows:

```

For each epoch:
   Set model to training mode
   Send model to device (GPU) using .to(device)
   For each batch in training dataloader:
     Get inputs and labels from batch. Send inputs to GPU device. 

     Zero out gradients
     Forward pass through model
     Calculate loss
     Backward pass
     Update weights
   
   Set model to evaluation mode - remember, no backprop/gradient calculation!
   For each batch in validation dataloader:
     Get inputs and labels from batch
     Forward pass through model
     Calculate validation loss
   
   Store training and validation metrics. Print them if you'd like. 

```


In [ ]:
# FOR FASTER TRAINING, USE GPU IF AVAILABLE
device = torch.device('cuda' if torch.cuda.is_available(
) else 'mps' if torch.backends.mps.is_available() else 'cpu')

print(f"Using device: {device}")

In [ ]:
from tqdm import tqdm

# Initialize lists to store metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

model.to(device)

# Training loop
def train_model(model, training_loader, validation_loader, optimizer, loss_fn, device, EPOCHS):
    # Initialize lists to store metrics
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    model.to(device)

    # Training loop
    for epoch in tqdm(range(EPOCHS), desc="Epochs"):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Training phase
        for inputs, labels in training_loader:
            inputs, labels = inputs.to(device), labels.to(device)
                        
            outputs = model(inputs)  # Forward pass

            # YOUR CODE HERE
            ...
            
            # Calculate accuracy
            predicted = torch.argmax(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(training_loader)
        epoch_acc = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        
        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = ...
                loss = ...
                
                val_loss += loss.item()

                #_, predicted = torch.max(outputs.data, 1)
                predicted = torch.argmax(outputs, 1)
                # print(f"predicted: {predicted}")
                # print(f"labels: {labels}")
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(validation_loader)
        val_acc = 100 * correct / total
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        print(f'Epoch [{epoch+1}/{EPOCHS}]')
        print(f'Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        print('-' * 60)
    
    return train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
EPOCHS = ...

train_losses, val_losses, train_accuracies, val_accuracies = train_model(model, 
                                                                         training_loader, 
                                                                         validation_loader, 
                                                                         optimizer, 
                                                                         loss_fn, 
                                                                         device, 
                                                                         EPOCHS)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 5**: Plotting model performance. 

Create two plots: 
- Training loss against epochs
- Validation accuracy epochs

Make sure your plots are appropriately labeled. 

In [ ]:
...


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Compare the loss and accuracy plots for training versus validation. What do you notice?

(Replace this text with your answer.)

<!-- END QUESTION -->



## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)